In [71]:
from fileoperations.fileoperations import getFileNamesInDir
from symbtrdataextractor import symbtrreader, extractor
import os
import json
import pandas as pd 

import pdb

In [72]:
symbTr = ''
symbtrtxtfile = os.path.join('../txt', symbTr + '.txt')
symbtrmu2file = os.path.join('../mu2', symbTr + '.mu2')

usuldict = json.load(open('./unittests/data/usul_extended.json','r'))

savefile = os.path.join('..', '..', 'symbtr_corrections', 'txt', symbTr + '.txt')

In [73]:
# extract symbtr data
txtdata = extractor.extract(symbtrtxtfile)[0]
mu2header = symbtrreader.readMu2Header(symbtrmu2file)[0]

if 'work' in txtdata.keys():
    mu2header['work'] = {'mu2_title':mu2header.pop('mu2_title')}
elif 'recording' in txtdata.keys():
    mu2header['recording'] = {'mu2_title':mu2header.pop('mu2_title')}
else:
    mu2header.pop('mu2_title')

# merge
data = extractor.merge(txtdata, mu2header)

    Some measures are skipped by the offsets
[15.0625, 17.0625, 19.0625, 21.0625, 23.0625, 24.01563, 25.01563, 26.01563, 27.01563, 28.01563, 29.01563, 30.01563, 31.01563, 32.01563, 33.01563, 34.01563, 35.01563, 36.01563, 37.01563, 38.01563, 39.01563, 40.01563, 41.01563, 42.01563, 43.01563, 44.01563, 45.01563, 46.01563, 47.01563, 48.01563, 49.01563, 50.01563, 51.01563, 52.14063, 53.01563, 54.26563, 55.01563, 56.64063]
kurdilihicazkar--sarki--duyek--her_kimde--civan_aga, Missing section info in lyrics.


In [74]:
for usul in usuldict.values():
    for uv in usul['variants']:
        if uv['mu2_name'] == data['usul']['mu2_name']:
            mertebe = uv['mertebe']
            zaman = uv['num_pulses']
            break


In [75]:
df = pd.read_csv(symbtrtxtfile, sep='\t')
for index, row in df.iterrows():
    # recompute the erroneous gracenotes with non-zero duration
    if row['Kod'] == 8 and row['Ms'] > 0:
        row['Pay'] = 0
        row['Payda'] = 0
        row['Ms'] = 0
        
    # recompute zaman and mertebe, if we hit kod 51
    if row['Kod'] == 51:
        zaman = row['Pay']
        mertebe = row['Payda']
        
    # compute offset
    offset_incr = 0 if row['Payda'] ==0 else float(row['Pay'])/row['Payda']*mertebe/zaman
    if index == 0:
        row['Offset'] = offset_incr
    else:
        prev_row = df.iloc[index-1]
        row['Offset'] = offset_incr + prev_row['Offset']
                
    # change null to empty string
    for key, val in row.iteritems():
        if pd.isnull(val):
            row[key] = ''
            
    # make sure that "Sira" column continues consecutively
    row['Sira'] = index+1
    
    # reassign
    df.iloc[index] = row
    

In [76]:
df.to_csv(savefile, sep='\t', index=False)

In [77]:
txtdata = extractor.extract(savefile)[0]

    Some measures are skipped by the offsets
[24.015625, 25.015625, 26.015625, 27.015625, 28.015625, 29.015625, 30.015625, 31.015625, 32.015625, 33.015625, 34.015625, 35.015625, 36.015625, 37.015625, 38.015625, 39.015625, 40.015625, 41.015625, 42.015625, 43.015625, 44.015625, 45.015625, 46.015625, 47.015625, 48.015625, 49.015625, 50.015625, 51.015625, 52.015625, 53.015625, 54.015625, 55.015625]
kurdilihicazkar--sarki--duyek--her_kimde--civan_aga, Missing section info in lyrics.
